In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

* READING DATA FRAMES

In [2]:
# Read the dataset
df1 = pd.read_csv("labeled_data.csv")

In [3]:
df1.drop_duplicates(subset=['title','text'], inplace=True)

In [4]:
df1.describe()

,title,text,publishing_date,source_num,label
count,373515,373515,373515,373515,373515
unique,363736,362821,285062,103,2
top,نصاف بن علية تتلقّى الجرعة الأولى من لقاح فايزر,احترام خصوصيتك هو أولويتنا\n\nنحن نستخدم ملفات...,2021-05-19T00:00:00,source_65,credible
freq,6,797,845,33706,207018


In [5]:
# Show the head of the DataFrame
print(df1.head())

                                               title  \
0  المنتخب الوطني المغربي لأقل من 20 سنة يخوض تجم...   
1  وزير النقل اعمارة:السرعة عامل مسبب لحوادث السي...   
2    ميسي يؤكد أن الفوز بكأس اسبانيا كان "نقطة تحول"   
3  دبلوماسي مغربي سابق بإسبانيا لـ2M.ma: خطأ مدري...   
4  شركتان، فرنسية وبريطانية تعلنان عن نتائج إيجاب...   

                                                text      publishing_date  \
0  يخوض المنتخب الوطني المغربي لكرة القدم لأقل من...  2021-05-23T00:00:00   
1  ترأس عبد القادر اعمارة، وزير التجهيز و النقل و...  2021-05-22T00:00:00   
2  أكد النجم الارجنتيني ليونيل ميسي أن فوز فريقه ...  2021-05-22T00:00:00   
3  أكّد الدبلوماسي المغربي السابق بإسبانيا، عبد ا...  2021-05-22T00:00:00   
4  أعلنت شركتا سانوفي الفرنسية العملاقة في تصنيع ...  2021-05-17T00:00:00   

  source_num     label  
0   source_1  credible  
1   source_1  credible  
2   source_1  credible  
3   source_1  credible  
4   source_1  credible  


In [6]:
#label
distinct_label = df1['label'].unique()
print("Distinct label:", distinct_label)

Distinct label: ['credible' 'not credible']


In [7]:
#preps for merg
df1['label'] = df1['label'].map({'credible': 'real', 'not credible': 'fake'})

In [8]:
# Read Arabic stop words from text file
with open(r"K:\STUDIES\----- M1 AI\S2\NLP\NLP-LAB\LAB 03\MODELS\Arabic stop words .txt", encoding="utf-8") as file:
    arabic_stopwords = set(file.read().splitlines())

In [9]:
""" 
# Getting the Arabic stopwords from NLTK
stop_words = set(stopwords.words('arabic'))
"""

" \n# Getting the Arabic stopwords from NLTK\nstop_words = set(stopwords.words('arabic'))\n"

In [10]:
# Function to clean the text
def clean_text(text):
    # Remove links
    text = re.sub(r'http\S+', '', text)
    
    # Remove emojis and punctuations
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove specific patterns like ":\n\n1"
    text = re.sub(r':\n\n\d+', '', text)
    
    # Remove empty spaces
    text = text.strip()
    
    return text

# Clean the text column
df1['cleaned_text'] = df1['text'].apply(clean_text)


# Function to remove Arabic stop words
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in arabic_stopwords]
    return ' '.join(filtered_words)

# Remove Arabic stop words
df1['cleaned_text'] = df1['cleaned_text'].apply(remove_stopwords)

# Show the head of the DataFrame with cleaned text
print(df1[['text', 'cleaned_text']].head())


                                                text  \
0  يخوض المنتخب الوطني المغربي لكرة القدم لأقل من...   
1  ترأس عبد القادر اعمارة، وزير التجهيز و النقل و...   
2  أكد النجم الارجنتيني ليونيل ميسي أن فوز فريقه ...   
3  أكّد الدبلوماسي المغربي السابق بإسبانيا، عبد ا...   
4  أعلنت شركتا سانوفي الفرنسية العملاقة في تصنيع ...   

                                        cleaned_text  
0  يخوض المنتخب الوطني المغربي لكرة القدم لأقل 20...  
1  ترأس عبد القادر اعمارة وزير التجهيز النقل اللو...  
2  أكد النجم الارجنتيني ليونيل ميسي فوز فريقه برش...  
3  أكد الدبلوماسي المغربي بإسبانيا عبد الخالق أطر...  
4  أعلنت شركتا سانوفي الفرنسية العملاقة تصنيع الأ...  


In [11]:
# Show the head of the DataFrame
print(df1.head())
#pd.set_option('display.max_colwidth', None)

                                               title  \
0  المنتخب الوطني المغربي لأقل من 20 سنة يخوض تجم...   
1  وزير النقل اعمارة:السرعة عامل مسبب لحوادث السي...   
2    ميسي يؤكد أن الفوز بكأس اسبانيا كان "نقطة تحول"   
3  دبلوماسي مغربي سابق بإسبانيا لـ2M.ma: خطأ مدري...   
4  شركتان، فرنسية وبريطانية تعلنان عن نتائج إيجاب...   

                                                text      publishing_date  \
0  يخوض المنتخب الوطني المغربي لكرة القدم لأقل من...  2021-05-23T00:00:00   
1  ترأس عبد القادر اعمارة، وزير التجهيز و النقل و...  2021-05-22T00:00:00   
2  أكد النجم الارجنتيني ليونيل ميسي أن فوز فريقه ...  2021-05-22T00:00:00   
3  أكّد الدبلوماسي المغربي السابق بإسبانيا، عبد ا...  2021-05-22T00:00:00   
4  أعلنت شركتا سانوفي الفرنسية العملاقة في تصنيع ...  2021-05-17T00:00:00   

  source_num label                                       cleaned_text  
0   source_1  real  يخوض المنتخب الوطني المغربي لكرة القدم لأقل 20...  
1   source_1  real  ترأس عبد القادر اعمارة وزير التجهيز 

In [12]:
df1.describe()

,title,text,publishing_date,source_num,label,cleaned_text
count,373515,373515,373515,373515,373515,373515
unique,363736,362821,285062,103,2,360433
top,نصاف بن علية تتلقّى الجرعة الأولى من لقاح فايزر,احترام خصوصيتك هو أولويتنا\n\nنحن نستخدم ملفات...,2021-05-19T00:00:00,source_65,real,احترام خصوصيتك أولويتنا نستخدم ملفات تعريف الا...
freq,6,797,845,33706,207018,797


In [13]:
df2 = pd.read_csv("cleaned_merged_data.csv")

In [14]:
df2.head()

,Title,Label,Topic,Origine,Article_date,Article_content,Article_correction,Article_content_cleaned,Article_correction_cleaned
0,54 عاما في القضاء.. نهاية حزينة لمعركة مقدسي ح...,real,politics,df1,NaN,القدس المحتلة- لن يتمكن المقدسي سامي درويش في ...,NaN,القدس المحتلة- لن يتمكن المقدسي سامي درويش في ...,NaN
1,خبراء إيرانيون يشرحون لماذا لا تستعجل طهران با...,real,politics,df1,NaN,طهران- منذ الهجوم الإسرائيلي على القنصلية الإي...,NaN,طهران- منذ الهجوم الإسرائيلي على القنصلية الإي...,NaN
2,فلسطيني يقود فريق خبراء بإيطاليا لبناء قضية إب...,real,politics,df1,NaN,غادر المحامي الفلسطيني راجي صوراني قطاع غزة رف...,NaN,غادر المحامي الفلسطيني راجي صوراني قطاع غزة رف...,NaN
3,مستوطنون يهاجمون قرى نابلس بحماية من جنود الاح...,real,politics,df1,NaN,نابلس- لليوم الثاني على التوالي، تتعرض قرية دو...,NaN,نابلس- لليوم الثاني على التوالي، تتعرض قرية دو...,NaN
4,جدل قانوني حول ترشح جاكوب زوما لانتخابات جنوب ...,real,politics,df1,NaN,بريتوريا- تقدمت اللجنة المستقلة للانتخابات في ...,NaN,بريتوريا- تقدمت اللجنة المستقلة للانتخابات في ...,NaN


In [15]:
df2.describe()

,Title,Label,Topic,Origine,Article_date,Article_content,Article_correction,Article_content_cleaned,Article_correction_cleaned
count,5734,5735,938,5735,5568,5735,2225,5735,2225
unique,5734,2,10,2,2927,5717,2225,5717,2225
top,54 عاما في القضاء.. نهاية حزينة لمعركة مقدسي ح...,fake,sport,df1,"May. 19, 2021",['الإدعاء'],\nبالبحث المباشر في موقع “وزارة الصحة والعمل و...,['الإدعاء'],بالبحث المباشر في موقع “وزارة الصحة والعمل وال...
freq,1,3115,336,5721,22,11,1,11,1


In [16]:
df2.drop_duplicates(subset=['Title','Article_content'], inplace=True)

In [17]:
df2.describe()

,Title,Label,Topic,Origine,Article_date,Article_content,Article_correction,Article_content_cleaned,Article_correction_cleaned
count,5734,5735,938,5735,5568,5735,2225,5735,2225
unique,5734,2,10,2,2927,5717,2225,5717,2225
top,54 عاما في القضاء.. نهاية حزينة لمعركة مقدسي ح...,fake,sport,df1,"May. 19, 2021",['الإدعاء'],\nبالبحث المباشر في موقع “وزارة الصحة والعمل و...,['الإدعاء'],بالبحث المباشر في موقع “وزارة الصحة والعمل وال...
freq,1,3115,336,5721,22,11,1,11,1


* MERGING THE DATAFRAMES

In [18]:
# Merge the dataframes on their titles
merged_df = pd.merge(df1, df2, left_on='title', right_on='Title', how='left')

# Select the required columns
final_df = merged_df[['cleaned_text', 'label', 'Article_correction_cleaned']]

# Rename the columns for consistency
final_df.columns = ['text', 'label', 'correction']

In [19]:
# Display the final dataframe
final_df.head()

,text,label,correction
0,يخوض المنتخب الوطني المغربي لكرة القدم لأقل 20...,real,NaN
1,ترأس عبد القادر اعمارة وزير التجهيز النقل اللو...,real,NaN
2,أكد النجم الارجنتيني ليونيل ميسي فوز فريقه برش...,real,NaN
3,أكد الدبلوماسي المغربي بإسبانيا عبد الخالق أطر...,real,NaN
4,أعلنت شركتا سانوفي الفرنسية العملاقة تصنيع الأ...,real,NaN


In [20]:
final_df_shuffled = final_df.sample(frac=1).reset_index(drop=True)

In [21]:
final_df_shuffled.head()

,text,label,correction
0,ارتفع القتلى جراء سقوط قذائف صاروخية أطلقتها ق...,fake,NaN
1,تمكن فريق الزمالك تحقيق فوزا بهدفين رد مولودية...,fake,NaN
2,سمير الوافي صفحته بفايسبوك مرة أخرى تستهدفني ع...,real,NaN
3,أعلن رئيس كتلة حزب قلب تونس بمجلس نواب الشعب أ...,real,NaN
4,علمت الراي مصادر امنية يمنع منعا نهائيا خروج ا...,fake,NaN


In [22]:
# 'label' column contains the labels in the DataFrame
final_df_shuffled['label'] = final_df_shuffled['label'].map({'fake': 0, 'real': 1})

In [23]:
# Replace empty strings with NaN
final_df_shuffled['label'].replace('', np.nan, inplace=True)

# Drop rows with NaN in the 'label' column
final_df_shuffled.dropna(subset=['label'], inplace=True)

In [24]:
final_df_shuffled.drop(columns=['correction'], inplace=True)


In [25]:
final_df_shuffled.head()

,text,label
0,ارتفع القتلى جراء سقوط قذائف صاروخية أطلقتها ق...,0
1,تمكن فريق الزمالك تحقيق فوزا بهدفين رد مولودية...,0
2,سمير الوافي صفحته بفايسبوك مرة أخرى تستهدفني ع...,1
3,أعلن رئيس كتلة حزب قلب تونس بمجلس نواب الشعب أ...,1
4,علمت الراي مصادر امنية يمنع منعا نهائيا خروج ا...,0


In [26]:
final_df_shuffled = final_df_shuffled.iloc[:10000]

# MAKING THE BI-LSTM MODEL

In [27]:
# Data Preparation
X = merged_df['text']
y = merged_df['label']

In [28]:
"""
# Remove Arabic stopwords
X = X.apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))
"""

"\n# Remove Arabic stopwords\nX = X.apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))\n"

In [29]:
# Use LabelEncoder to convert labels to numerical format
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [30]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [31]:
# Tokenization and Padding
max_vocab = 10000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)

In [32]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

KeyboardInterrupt: 

In [ ]:
max_sequence_length = 128   # Set an appropriate value
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
# LSTM Model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))  # Dropout layer for regularization
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(units=len(np.unique(y_encoded)), activation='softmax'))

In [ ]:
# Compile the Model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# Print the Model Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 128, 100)          134204900 
                                                                 
 spatial_dropout1d_1 (Spati  (None, 128, 100)          0         
 alDropout1D)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 128, 64)           42240     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 134280294 (512.24 MB)
Trainable params: 134280294 (512.24 MB)
Non-trainable params: 0 (0.00 Byte)
________

In [ ]:
# Train the Model
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(X_train_padded, y_train, epochs=10, validation_split=0.2, batch_size=256, shuffle=True, callbacks=[early_stop]) #batch_size=64

Epoch 1/10
  58/7471 [..............................] - ETA: 2:54:51 - loss: 0.6922 - accuracy: 0.5399

In [ ]:
# Evaluate the Model
y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
# Evaluate other metrics
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()